In [1]:
using CSV, DataFrames
using Microgrids
using NLopt # optimization solvers
using Printf # pretty print results
using PyPlot
using PyCall
using BenchmarkTools # for timing performance, optional
np=pyimport("numpy")


PyObject <module 'numpy' from 'C:\\Users\\nikiemaje\\.julia\\conda\\3\\x86_64\\lib\\site-packages\\numpy\\__init__.py'>

In [2]:
with_plots = isdefined(Main, :PyPlot)

true

In [3]:
td = collect((0:nsteps-1)/24) 

LoadError: UndefVarError: `nsteps` not defined

In [4]:
mg= default_microgrid()

Microgrid{Float64}(Project(25, 0.05, 1.0, "€"), [1453.0, 1331.0, 1214.0, 1146.0, 1078.0, 1035.0, 1032.0, 1072.0, 1142.0, 1208.0  …  1479.0, 1508.0, 1410.0, 1370.0, 1467.0, 1587.0, 1561.0, 1472.0, 1533.0, 1483.0], DispatchableCompound{Float64}(ProductionUnit{Float64}[ProductionUnit{Float64}(2000.0, 0.0, 0.24, 1.0, 400.0, 0.02, 15000.0, 0.0, 1.0, 1.0, "L", "KWh")], ProductionUnit{Float64}[ProductionUnit{Float64}(1800.0, 0.0, 0.0625, 0.0, 1000.0, 10.0, 45000.0, 0.05, 1.0, 1.0, "Kg", "KWh")]), ProductionUnit{Float64}[ProductionUnit{Float64}(2000.0, 0.0, 56.0, 0.0, 1000.0, 10.0, 45000.0, 0.05, 1.0, 1.0, "KWh", "Kg")], TankCompound{Float64}(Tank{Float64}(0.0, 0.0, 0.0, 25.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0), Tank{Float64}(10000.0, 0.8, 0.0025, 25.0, 0.0, 0.0, 0.1, 1.0, 1.0, 1.0)), Battery{Float64}(5000.0, 350.0, 10.0, 15.0, 3000.0, 1.0, 1.0, 0.05, 0.0, 0.0, 1.0, 1.0, 1.0), NonDispatchableSource[Photovoltaic{Float64}(3000.0, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.000000000000001e-5  …  

In [5]:
"""Simulate the performance of a Microgrid project of size `x`
with x=[power_rated_gen, energy_rated_sto, power_rated_pv, power_rated_wind]

Returns mg, traj, stats, costs
"""
    function simulate_microgrid(x,dispatch)
    # Split decision variables (converted MW → kW):
   
    mg = new_microgrid(x,dispatch)
    # Launch simulation:
    traj, stats, costs = simulate(mg,dispatch)

    return  traj ,stats, costs
end

simulate_microgrid

In [6]:
 # Baseline sizing: same as in Microgrid_Wind-Solar.ipynb notebook
x_base = [energy_rated_sto, power_rated_pv,power_rated_wind, power_rated_elyz,power_rated_fc ,capacity_rated_hytank]/1000.
# run simulation:
traj, stats, costs = simulate_microgrid(x_base,dispatch_2)
x_base, costs.lcoe, costs.npc/1e6

([5.0, 3.0, 1.8, 2.0, 1.8, 10.0], 0.26345419104330475, 23.59720165769015)

In [7]:


function plot_oper_traj(mg, oper_traj)
 
     z=zeros(Float64,12)
tm=zeros(Float64,13)
tm[13]=8760/24
 for i=1:12  
    z[i]= oper_traj.LoH[i*730+1]
    tm[i]=(i-1)*730/24 
end
    fig,( ax1, ax2) = plt.subplots(2,1)
    y1 = min.(oper_traj.Prenew_pot,mg.load)
    y2 = max.( oper_traj.Pbatt,0.0)
    y3 = oper_traj.Pfc
    y4 = abs.(min.( oper_traj.Pbatt,0.0))
    y5 = oper_traj.Pelyz
    y6 = oper_traj.power_shedding
    y7 = oper_traj.power_curtailment
    
 
        y=np.vstack([y1,y2,y3,y4,y5,y6,y7])
        fieldNames =(["Pren used by the load","battery_discharge","fuel cell","battery_charge", "elyz","shedding","spillage"])
    
    fieldColors = (["orange","green","pink","green","magenta","black","red"])
         
    
    ax1.plot(td, oper_traj.Prenew_pot,lw=2, "tab:orange", label="ren")
    ax1.plot(td, mg.load,lw=2, label="load")
    stacks = ax1.stackplot(td,y,labels=fieldNames, colors = fieldColors,alpha=0.8)
    ax2.plot(td, oper_traj.LoH[1:end-1]/mg.tanks.h2Tank.capacity, "C2")
    stacks[2].set_hatch("-")
    stacks[4].set_hatch("+")
    stacks[6].set_hatch("\\")   
    stacks[7].set_hatch("\\") 
    
    ax3=ax1.twinx()
    ax3.set_ylim(0, 2)
    ax3.plot(td, oper_traj.Ebatt[1:end-1]/mg.storage.energy_rated, ls="--","C2",label="hourly battery level")
    ax2.plot(td, oper_traj.LoH[1:end-1], "C2",label="hourly H2 level")
    
    ax2.stairs(z,tm,fill="True",label="level of H2 at the end of each month")
         
    
    ax1.set_title("hourly power repartition")
    ax2.set_title("H2 level")
    ax1.legend()
    ax1.grid(true)
    ax1.set(ylabel="kW")
    ax2.legend()
    ax3.legend()
    ax2.grid(true)
    ax2.set(ylabel="kg")
    
       fig.tight_layout()


   
     return fig, ax1, ax2
    
end


plot_oper_traj (generic function with 1 method)

In [8]:
Pload_max = maximum(Pload)

xmin = [0., 1e-3, 0., 0., 0., 0.] # 1e-3 instead of 0.0, because LCOE is NaN if ther is exactly zero generation
x0 = [3.0, 3.0, 2.0, 1.0, 1.0, 5.0] * (Pload_max/1000)
xmax = [10.0, 15.0, 5.0, 20., 1.2 , 20. ] * (Pload_max/1000)

6-element Vector{Float64}:
 17.07
 25.605
  8.535
 34.14
  2.0484
 17.07

In [9]:
"""Optimize sizing of microgrid based on the `obj` function

Parameters:
- `x0`: initial sizing (for the algorithms which need them)
- `shed_max`: load shedding penalty threshold (same as in `obj`)
- `algo` could be one of LN_SBPLX, GN_DIRECT, GN_ESCH...
- `maxeval`: maximum allowed number of calls to the objective function,
  that is to the microgrid simulation
- `xtol_rel`: termination condition based on relative change of sizing, see NLopt doc.
- `srand`: random number generation seed (for algorithms which use some stochastic search)

Problem bounds are taken as the global variables `xmin`, `xmax`,
but could be added to the parameters as well.
"""
function optim_mg(x0, shed_max,dispatch, algo=:LN_SBPLX, maxeval=1000, xtol_rel=1e-8, srand=1)
    nx = length(x0) # number of optim variables
    opt = Opt(algo, nx)
    NLopt.srand(srand)
    
    opt.lower_bounds = xmin
    opt.upper_bounds = xmax
    opt.min_objective = (x, grad) -> obj(x, grad,dispatch, shed_max)
    opt.xtol_rel = xtol_rel
    opt.maxeval = maxeval
    
    (fopt, xopt, ret) = optimize(opt, x0)
    return xopt, ret, opt.numevals
end

optim_mg

In [11]:
"Multi-objective criterion for microgrid performance: lcoe, shedding rate"
function obj_multi(x,dispatch)
   traj,stats, costs = simulate_microgrid(x,dispatch)
    # Extract KPIs of interest
    lcoe = costs.lcoe # $/kWh
    shed_rate = stats.shed_rate; # in [0,1]
    return lcoe, shed_rate
end

obj_multi

In [12]:
function obj(x, grad,dispatch, shed_max, w_shed_max=1e5)
    lcoe, shed_rate = obj_multi(x,dispatch)
    over_shed = shed_rate - shed_max
    if over_shed > 0.0
        penalty = w_shed_max*over_shed
    else
        penalty = 0.0
    end
    J = lcoe + penalty
end

obj (generic function with 2 methods)

In [50]:
algo = :GN_CRS2_LM # could be one of GN_CRS2_LM, GN_DIRECT, GN_ESCH, LN_SBPLX...
shed_max = 0.0001 # in [0,1]
maxeval=100000
xopt, ret, numevals = optim_mg(x0, shed_max,dispatch_1, algo, maxeval)

@printf("%s algo: %s after %d iterations. \nx*=", algo, ret, numevals)
println(round.(xopt*1000; digits=1)) # kW
lcoe_opt, shed_rate_opt = obj_multi(xopt,dispatch_1)
println("LCOE*: ", lcoe_opt)
println("shed*: ", shed_rate_opt)

GN_CRS2_LM algo: XTOL_REACHED after 69177 iterations. 
x*=[6750.3, 4362.3, 2143.2, 887.7, 1495.3, 12606.8]
LCOE*: 0.25620326210747557
shed*: 9.999998227674223e-5


In [42]:
mg, traj, stats, costs = simulate_microgrid(xopt,dispatch_1)
stats

OperationStats with fields:
- served_energy: 6.7682e6 kWh
- shed_energy: 6775.0 kWh
- shed_max: 256.54 kW
- shed_hours: 141.0 h
- shed_duration_max: 8.0 h
- shed_rate: 0.001 in [0,1]
- gen_energy: 0.0 kWh
- gen_hours: 0.0 h
- gen_fuel: 0.0 L
- fc_energy: 983780.0 kWh
- fc_hours: 1829.0 h
- h2_consumed: 61486.0 kg
- h2_produced: 64898.0 kg
- elyz_hours: 4051.0 h
- elyz_consumed_energy: 3.6343e6 kWh
- h2_chain_loss: 2.6503e6 kWh
- storage_cycles: 154.64 
- storage_char_energy: 483310.0 kWh
- storage_dis_energy: 440120.0 kWh
- storage_loss_energy: 46172.0 kWh
- spilled_energy: 785770.0 kWh
- spilled_max: 3136.2 kW
- spilled_rate: 0.076678 in [0,1]
- renew_potential_energy: 1.0248e7 kWh
- renew_energy: 9.4619e6 kWh
- renew_rate: 1.0 in [0,1]


In [46]:
plot_oper_traj(mg,traj)

(Figure(PyObject <Figure size 800x600 with 3 Axes>), PyObject <Axes: title={'center': 'hourly power repartition'}, ylabel='kW'>, PyObject <Axes: title={'center': 'H2 level'}, ylabel='kg'>)

In [6]:
algo = :GN_CRS2_LM # could be one of GN_CRS2_LM, GN_DIRECT, GN_ESCH, LN_SBPLX...
shed_max = 0.001 # in [0,1]
maxeval=100000
xopt2, ret, numevals = optim_mg(x0, shed_max,dispatch_2, algo, maxeval)

@printf("%s algo: %s after %d iterations. \nx*=", algo, ret, numevals)
println(round.(xopt2*1000; digits=1)) # kW
lcoe_opt, shed_rate_opt = obj_multi(xopt2,dispatch_2)
println("LCOE*: ", lcoe_opt)
println("shed*: ", shed_rate_opt)

LoadError: UndefVarError: `optim_mg` not defined

In [48]:
mg2, traj2, stats2, costs2 = simulate_microgrid(xopt2,dispatch_2)
stats2

OperationStats with fields:
- served_energy: 6.7682e6 kWh
- shed_energy: 6775.0 kWh
- shed_max: 1203.8 kW
- shed_hours: 65.0 h
- shed_duration_max: 11.0 h
- shed_rate: 0.001 in [0,1]
- gen_energy: 0.0 kWh
- gen_hours: 0.0 h
- gen_fuel: 0.0 L
- fc_energy: 1.3541e6 kWh
- fc_hours: 2843.0 h
- h2_consumed: 84630.0 kg
- h2_produced: 87968.0 kg
- elyz_hours: 5421.0 h
- elyz_consumed_energy: 4.9262e6 kWh
- h2_chain_loss: 3.5719e6 kWh
- storage_cycles: 46.307 
- storage_char_energy: 8484.3 kWh
- storage_dis_energy: 7844.2 kWh
- storage_loss_energy: 816.42 kWh
- spilled_energy: 534940.0 kWh
- spilled_max: 1974.7 kW
- spilled_rate: 0.049186 in [0,1]
- renew_potential_energy: 1.0876e7 kWh
- renew_energy: 1.0341e7 kWh
- renew_rate: 1.0 in [0,1]


In [5]:
plot_oper_traj(mg2,traj2)

LoadError: UndefVarError: `plot_oper_traj` not defined